## 분석

### 데이터 로드

In [2]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/final/'
web = pd.read_csv(path + 'dataset_genre.csv')

In [3]:
# 컬럼 정리
web=web[['title','writer','illustrator','day','genre','story','platform']]

In [4]:
web.head(3)

,title,writer,illustrator,day,genre,story,platform
0,참교육,채용택,한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용...,네이버 웹툰
1,신의 탑,SIU,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑,네이버 웹툰
2,뷰티풀 군바리,설이,윤성원,월,드라마,여자도 군대에 간다면? 본격 여자도 군대 가는 만화!,네이버 웹툰


In [13]:
import numpy as np
matrics_path = '/content/drive/MyDrive/Colab Notebooks/final/matrics/'

okt_matrices = np.load(matrics_path+'okt_matrics_전처리후.npy',allow_pickle=True)
kkma_matrices = np.load(matrics_path+'kkma_matrics_전처리후.npy',allow_pickle=True)
hannanum_matrices = np.load(matrics_path+'hannanum_matrics_전처리후.npy',allow_pickle=True)
komoran_matrices = np.load(matrics_path+'komoran_matrics_전처리후.npy',allow_pickle=True)

In [10]:

from scipy.sparse import hstack

def hstack_matrices(matrices, noauthor, w_author, w_genre):
    cnt_genre_matrix, lb_writer_matrix, lb_illustrator_matrix, tfidf_story_matrix = matrices
    
    # tfidf_title_matrix,
    # 만들어진 행렬을 합치기 위한 리스트
    sparse_matrix_list = [cnt_genre_matrix * w_genre, tfidf_story_matrix]
    if not noauthor:
        sparse_matrix_list.append(lb_writer_matrix * w_author)
        sparse_matrix_list.append(lb_illustrator_matrix * w_author)
    
    # if not notitle:
    #     sparse_matrix_list.append(tfidf_title_matrix)
    
    sparse_matrix_list = tuple(sparse_matrix_list)

    # 행렬 결합
    X_features_sparse = hstack(sparse_matrix_list).tocsr()
#     print(type(X_features_sparse), X_features_sparse.shape)
    return X_features_sparse

In [14]:
# 웹툰 제목을 입력하면 해당 인덱스를 반환해줄 Series 생성
indices = pd.Series(web.index, index=web['title'])
indices.head(3)

title
참교육        0
신의 탑       1
뷰티풀 군바리    2
dtype: int64

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tokenizer_matrices = [okt_matrices, kkma_matrices, hannanum_matrices, komoran_matrices]
# notitle=False,
def get_recomm(title, rank=10, same_genre=False, noauthor=False,  w_author=0.7, w_genre=1.3, tokenizer_matrices=tokenizer_matrices):
    # 코사인 유사도를 담을 array 생성
    cosine_sim = np.zeros((len(web), len(web)))
    
    # 각 토큰화 도구로 만들어진 행렬을 합쳐서 코사인 유사도 생성
    for matrices in tokenizer_matrices:
        X_features_sparse = hstack_matrices(matrices, noauthor=noauthor,  w_author=w_author, w_genre=w_genre)
        cosine_sim += cosine_similarity(X_features_sparse, X_features_sparse)
    cosine_sim /= len(tokenizer_matrices)
    
    # 결과 출력을 위한 데이터 프레임 복사
    print_copy = web.copy()
    
    # 선택한 웹툰 제목으로부터 해당되는 인덱스를 받아옴
    idx = indices[title]
    
    # 인덱스로부터 장르를 뽑아옴
    genre = print_copy.loc[idx, 'genre']

    # 유사도를 DataFrame에 붙임
    print_copy['similarity'] = cosine_sim[idx]
    
    if same_genre:
        # 동일 장르만을 보여줄 경우
        return print_copy[print_copy['genre'] == genre].sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]
        
    else:
        # 다른 장르도 보여줄 경우
        return print_copy.sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]

## 가중치 테스트

### 패션왕

In [16]:
# w_author=0.7, w_genre=1.3
get_recomm('패션왕')

,title,writer,illustrator,day,genre,story,platform,similarity
653,회춘,기안84,기안84,완결,드라마,두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.735231
466,복학왕,기안84,기안84,완결,드라마,패션왕 우기명이 돌아왔다!즐거운 대학 생활?! 과연?!,네이버 웹툰,0.727520
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,드라마,신은 어디에나 있다,네이버 웹툰,0.500186
757,오라존미,허5파6,허5파6,완결,드라마,만화를 그리는 작가와 만화를 말하는 작가.,네이버 웹툰,0.480338
2174,커피와 하루 the start,한상훈,한상훈,완결,드라마,커피와 함께 살아가는 네 사람의 이야기,카카오웹툰,0.474650
1691,곱게 자란 자식,이무기,이무기,완결,드라마,평범한 시골 소녀 이야기. .소녀만 평범한 그런 잔인한 이야기,카카오웹툰,0.474191
1366,다욤이의 다이어트 다이어리,아키,심윤수,완결,드라마,다시! 다시! 내일 또 다시! 언제까지 다시어트 를 할겁니까? 무거운 이야기를 가볍...,네이버 웹툰,0.473679
1346,유토피아,늑대삼,늑대삼,완결,드라마,평범하지 않은 두 사람의 평범한 사랑 이야기. 외로운 두 사람의 전혀 외롭지 않은 ...,네이버 웹툰,0.473419
1878,애휴,조금산,조금산,완결,드라마,서로 잘 모르는 남녀 이야기,카카오웹툰,0.472921
2133,한여름밤의 꿈,김용회,김용회,완결,드라마,"숲, 소년 그리고 한여름의 꿈이 시작되는 순간의 이야기",카카오웹툰,0.471273


In [27]:
get_recomm('패션왕',w_author=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
653,회춘,기안84,기안84,완결,드라마,두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.792814
466,복학왕,기안84,기안84,완결,드라마,패션왕 우기명이 돌아왔다!즐거운 대학 생활?! 과연?!,네이버 웹툰,0.786780
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,드라마,신은 어디에나 있다,네이버 웹툰,0.383293
757,오라존미,허5파6,허5파6,완결,드라마,만화를 그리는 작가와 만화를 말하는 작가.,네이버 웹툰,0.371817
2174,커피와 하루 the start,한상훈,한상훈,완결,드라마,커피와 함께 살아가는 네 사람의 이야기,카카오웹툰,0.371421
1691,곱게 자란 자식,이무기,이무기,완결,드라마,평범한 시골 소녀 이야기. .소녀만 평범한 그런 잔인한 이야기,카카오웹툰,0.371062
1366,다욤이의 다이어트 다이어리,아키,심윤수,완결,드라마,다시! 다시! 내일 또 다시! 언제까지 다시어트 를 할겁니까? 무거운 이야기를 가볍...,네이버 웹툰,0.370661
1346,유토피아,늑대삼,늑대삼,완결,드라마,평범하지 않은 두 사람의 평범한 사랑 이야기. 외로운 두 사람의 전혀 외롭지 않은 ...,네이버 웹툰,0.370458
1878,애휴,조금산,조금산,완결,드라마,서로 잘 모르는 남녀 이야기,카카오웹툰,0.370068
2133,한여름밤의 꿈,김용회,김용회,완결,드라마,"숲, 소년 그리고 한여름의 꿈이 시작되는 순간의 이야기",카카오웹툰,0.368778


In [35]:
get_recomm('패션왕',w_author=1, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
653,회춘,기안84,기안84,완결,드라마,두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.757074
466,복학왕,기안84,기안84,완결,드라마,패션왕 우기명이 돌아왔다!즐거운 대학 생활?! 과연?!,네이버 웹툰,0.750000
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,드라마,신은 어디에나 있다,네이버 웹툰,0.269338
2174,커피와 하루 the start,한상훈,한상훈,완결,드라마,커피와 함께 살아가는 네 사람의 이야기,카카오웹툰,0.262991
1691,곱게 자란 자식,이무기,이무기,완결,드라마,평범한 시골 소녀 이야기. .소녀만 평범한 그런 잔인한 이야기,카카오웹툰,0.262570
1366,다욤이의 다이어트 다이어리,아키,심윤수,완결,드라마,다시! 다시! 내일 또 다시! 언제까지 다시어트 를 할겁니까? 무거운 이야기를 가볍...,네이버 웹툰,0.262100
1346,유토피아,늑대삼,늑대삼,완결,드라마,평범하지 않은 두 사람의 평범한 사랑 이야기. 외로운 두 사람의 전혀 외롭지 않은 ...,네이버 웹툰,0.261862
1878,애휴,조금산,조금산,완결,드라마,서로 잘 모르는 남녀 이야기,카카오웹툰,0.261405
2133,한여름밤의 꿈,김용회,김용회,완결,드라마,"숲, 소년 그리고 한여름의 꿈이 시작되는 순간의 이야기",카카오웹툰,0.259893
1898,다시피는 꽃,김용회,김용회,완결,드라마,"나이를 먹었든, 먹지 않았든 누구나 자신만의 이야기가 있다. 위안부 할머니들의 애틋...",카카오웹툰,0.259767


In [36]:
get_recomm('패션왕',noauthor=True, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,드라마,신은 어디에나 있다,네이버 웹툰,0.603553
757,오라존미,허5파6,허5파6,완결,드라마,만화를 그리는 작가와 만화를 말하는 작가.,네이버 웹툰,0.551777
2174,커피와 하루 the start,한상훈,한상훈,완결,드라마,커피와 함께 살아가는 네 사람의 이야기,카카오웹툰,0.525983
1691,곱게 자란 자식,이무기,이무기,완결,드라마,평범한 시골 소녀 이야기. .소녀만 평범한 그런 잔인한 이야기,카카오웹툰,0.525141
1366,다욤이의 다이어트 다이어리,아키,심윤수,완결,드라마,다시! 다시! 내일 또 다시! 언제까지 다시어트 를 할겁니까? 무거운 이야기를 가볍...,네이버 웹툰,0.524200
1346,유토피아,늑대삼,늑대삼,완결,드라마,평범하지 않은 두 사람의 평범한 사랑 이야기. 외로운 두 사람의 전혀 외롭지 않은 ...,네이버 웹툰,0.523724
1878,애휴,조금산,조금산,완결,드라마,서로 잘 모르는 남녀 이야기,카카오웹툰,0.522810
2133,한여름밤의 꿈,김용회,김용회,완결,드라마,"숲, 소년 그리고 한여름의 꿈이 시작되는 순간의 이야기",카카오웹툰,0.519785
1898,다시피는 꽃,김용회,김용회,완결,드라마,"나이를 먹었든, 먹지 않았든 누구나 자신만의 이야기가 있다. 위안부 할머니들의 애틋...",카카오웹툰,0.519534
1077,3인칭,꼬마비,애마비,완결,드라마,쉽지 않은 사랑 이야기.그녀를 찾아서..,네이버 웹툰,0.519522


In [23]:
get_recomm('패션왕',w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
653,회춘,기안84,기안84,완결,드라마,두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.819386
466,복학왕,기안84,기안84,완결,드라마,패션왕 우기명이 돌아왔다!즐거운 대학 생활?! 과연?!,네이버 웹툰,0.814126
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,드라마,신은 어디에나 있다,네이버 웹툰,0.195938
2174,커피와 하루 the start,한상훈,한상훈,완결,드라마,커피와 함께 살아가는 네 사람의 이야기,카카오웹툰,0.195533
1691,곱게 자란 자식,이무기,이무기,완결,드라마,평범한 시골 소녀 이야기. .소녀만 평범한 그런 잔인한 이야기,카카오웹툰,0.195220
1366,다욤이의 다이어트 다이어리,아키,심윤수,완결,드라마,다시! 다시! 내일 또 다시! 언제까지 다시어트 를 할겁니까? 무거운 이야기를 가볍...,네이버 웹툰,0.194870
1346,유토피아,늑대삼,늑대삼,완결,드라마,평범하지 않은 두 사람의 평범한 사랑 이야기. 외로운 두 사람의 전혀 외롭지 않은 ...,네이버 웹툰,0.194693
1878,애휴,조금산,조금산,완결,드라마,서로 잘 모르는 남녀 이야기,카카오웹툰,0.194353
2133,한여름밤의 꿈,김용회,김용회,완결,드라마,"숲, 소년 그리고 한여름의 꿈이 시작되는 순간의 이야기",카카오웹툰,0.193229
1898,다시피는 꽃,김용회,김용회,완결,드라마,"나이를 먹었든, 먹지 않았든 누구나 자신만의 이야기가 있다. 위안부 할머니들의 애틋...",카카오웹툰,0.193135


### 마음의소리

In [37]:
get_recomm('마음의소리')

,title,writer,illustrator,day,genre,story,platform,similarity
1454,취업의소리,조석,워니,완결,개그,"조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.594005
1042,우리들은 푸르다,문택수,문택수,완결,개그,그들과 함께라면 일상도 개그가 된다..! 범상치않은 그들이 모인 연파랑고 1학년 3...,네이버 웹툰,0.475701
1731,구리의 구리구리 컴백,구리,구리,완결,개그,"논픽션 좌충우돌, 돌아온 구리의 구리구리한 개그 일상툰",카카오웹툰,0.475232
1985,그놈은 여고생,백영민,백영민,완결,개그,최강 이라 불린 고등학생 최강배. 그가 치마를 입고 여학생이 된 이유는?,카카오웹툰,0.474317
2119,결혼해도 똑같네,네온비,네온비,완결,개그,"만화가 네온비, 캐러멜 부부 탄생! 그리고 이어지는 충격 개그 신화!",카카오웹툰,0.474050
1101,2016 학교 다녀오겠습니다,웹툰작가,웹툰작가,완결,개그,신학기가 외롭고 힘든 당신을 위해 준비했다웹툰작가들의 개그 단편선,네이버 웹툰,0.473038
2002,지구방위연구소,니나노머신,니나노머신,완결,개그,지구 수호의 임무를 띤 연구소에서 펼쳐지는 개그 퍼레이드!,카카오웹툰,0.472444
2241,까야제맛,하준성,하준성,완결,개그,체이서 하준성 작가의 무한 개그본능이 살아난다!,카카오웹툰,0.471410
1025,웃지 않는 개그반 시즌3,현용민,현용민,완결,개그,미친예술고 개그반에 저주가 내려졌다!,네이버 웹툰,0.471301
1249,키드갱 시즌2,신영우,신영우,완결,개그,키드갱 22권 그 이후! 정통 개그만화가 부활한다!,네이버 웹툰,0.470624


In [39]:
get_recomm('마음의소리',w_author=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1454,취업의소리,조석,워니,완결,개그,"조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.573561
1024,문유,조석,조석,완결,판타지,지구가 멸망했다. 나는 혼자 달에 있다. 마음의 소리 조석의 우주를 넘나드는 판타지...,네이버 웹툰,0.461863
899,조의 영역,조석,조석,완결,스릴러,물에 살던 녀석들의 모습이 상식을 벗어나기 시작했다. 마음의 소리 조석의 재난 스릴러,네이버 웹툰,0.451572
1157,조석축구만화,조석,조석,완결,스포츠,조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.449896
200,묵시의 인플루언서,조석,조석,목,스릴러,"어느 날 동네에 생긴 정체 모를 알 하나. 알은 빠른 속도로 퍼지더니, 급기야 알에...",네이버 웹툰,0.428051
1444,N의등대-busted,조석,조석,완결,스릴러,"동일한 세계의 색다른 4가지 이야기. 함께 볼 수도, 따로 볼 수도 있는 멀티 플롯...",네이버 웹툰,0.427038
594,후기,조석,조석,완결,스릴러,무가지에 연재되던 삼류 소설이 사실은 누군가의 살인 후기였다!,네이버 웹툰,0.426439
1331,자율공상축구탐구만화,조석,조석,완결,스포츠,"월드컵, 유럽리그, 국내리그 등 축구라면 가리지 않고 모두 그려보는 자율축구탐구만화...",네이버 웹툰,0.426439
462,행성인간,조석,조석,완결,스릴러,내가 어떤 존재들의 행성이라면?조석 작가가 그리는 신개념 스릴러,네이버 웹툰,0.426439
1042,우리들은 푸르다,문택수,문택수,완결,개그,그들과 함께라면 일상도 개그가 된다..! 범상치않은 그들이 모인 연파랑고 1학년 3...,네이버 웹툰,0.372244


In [38]:
get_recomm('마음의소리',w_author=1.0,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1024,문유,조석,조석,완결,판타지,지구가 멸망했다. 나는 혼자 달에 있다. 마음의 소리 조석의 우주를 넘나드는 판타지...,네이버 웹툰,0.541535
899,조의 영역,조석,조석,완결,스릴러,물에 살던 녀석들의 모습이 상식을 벗어나기 시작했다. 마음의 소리 조석의 재난 스릴러,네이버 웹툰,0.529468
1157,조석축구만화,조석,조석,완결,스포츠,조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.527504
200,묵시의 인플루언서,조석,조석,목,스릴러,"어느 날 동네에 생긴 정체 모를 알 하나. 알은 빠른 속도로 퍼지더니, 급기야 알에...",네이버 웹툰,0.501889
1444,N의등대-busted,조석,조석,완결,스릴러,"동일한 세계의 색다른 4가지 이야기. 함께 볼 수도, 따로 볼 수도 있는 멀티 플롯...",네이버 웹툰,0.500702
594,후기,조석,조석,완결,스릴러,무가지에 연재되던 삼류 소설이 사실은 누군가의 살인 후기였다!,네이버 웹툰,0.500000
1454,취업의소리,조석,워니,완결,개그,"조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.500000
1331,자율공상축구탐구만화,조석,조석,완결,스포츠,"월드컵, 유럽리그, 국내리그 등 축구라면 가리지 않고 모두 그려보는 자율축구탐구만화...",네이버 웹툰,0.500000
462,행성인간,조석,조석,완결,스릴러,내가 어떤 존재들의 행성이라면?조석 작가가 그리는 신개념 스릴러,네이버 웹툰,0.500000
1042,우리들은 푸르다,문택수,문택수,완결,개그,그들과 함께라면 일상도 개그가 된다..! 범상치않은 그들이 모인 연파랑고 1학년 3...,네이버 웹툰,0.263956


In [41]:
get_recomm('마음의소리',noauthor=True,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1042,우리들은 푸르다,문택수,문택수,완결,개그,그들과 함께라면 일상도 개그가 된다..! 범상치않은 그들이 모인 연파랑고 1학년 3...,네이버 웹툰,0.527912
1731,구리의 구리구리 컴백,구리,구리,완결,개그,"논픽션 좌충우돌, 돌아온 구리의 구리구리한 개그 일상툰",카카오웹툰,0.527051
1985,그놈은 여고생,백영민,백영민,완결,개그,최강 이라 불린 고등학생 최강배. 그가 치마를 입고 여학생이 된 이유는?,카카오웹툰,0.525372
2119,결혼해도 똑같네,네온비,네온비,완결,개그,"만화가 네온비, 캐러멜 부부 탄생! 그리고 이어지는 충격 개그 신화!",카카오웹툰,0.524881
1101,2016 학교 다녀오겠습니다,웹툰작가,웹툰작가,완결,개그,신학기가 외롭고 힘든 당신을 위해 준비했다웹툰작가들의 개그 단편선,네이버 웹툰,0.523025
2002,지구방위연구소,니나노머신,니나노머신,완결,개그,지구 수호의 임무를 띤 연구소에서 펼쳐지는 개그 퍼레이드!,카카오웹툰,0.521934
2241,까야제맛,하준성,하준성,완결,개그,체이서 하준성 작가의 무한 개그본능이 살아난다!,카카오웹툰,0.520038
1025,웃지 않는 개그반 시즌3,현용민,현용민,완결,개그,미친예술고 개그반에 저주가 내려졌다!,네이버 웹툰,0.519837
1249,키드갱 시즌2,신영우,신영우,완결,개그,키드갱 22권 그 이후! 정통 개그만화가 부활한다!,네이버 웹툰,0.518594
1303,이말년씨리즈,이말년,이말년,완결,개그,독특한 이야기 전개 방식을 확립한 이말년 작가가 선보이는 엉뚱한 상상과 황당한 설정...,네이버 웹툰,0.517306


In [40]:
get_recomm('마음의소리',w_author=1.3,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1024,문유,조석,조석,완결,판타지,지구가 멸망했다. 나는 혼자 달에 있다. 마음의 소리 조석의 우주를 넘나드는 판타지...,네이버 웹툰,0.659134
899,조의 영역,조석,조석,완결,스릴러,물에 살던 녀석들의 모습이 상식을 벗어나기 시작했다. 마음의 소리 조석의 재난 스릴러,네이버 웹툰,0.650162
1157,조석축구만화,조석,조석,완결,스포츠,조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.648701
200,묵시의 인플루언서,조석,조석,목,스릴러,"어느 날 동네에 생긴 정체 모를 알 하나. 알은 빠른 속도로 퍼지더니, 급기야 알에...",네이버 웹툰,0.629658
1444,N의등대-busted,조석,조석,완결,스릴러,"동일한 세계의 색다른 4가지 이야기. 함께 볼 수도, 따로 볼 수도 있는 멀티 플롯...",네이버 웹툰,0.628775
594,후기,조석,조석,완결,스릴러,무가지에 연재되던 삼류 소설이 사실은 누군가의 살인 후기였다!,네이버 웹툰,0.628253
462,행성인간,조석,조석,완결,스릴러,내가 어떤 존재들의 행성이라면?조석 작가가 그리는 신개념 스릴러,네이버 웹툰,0.628253
1331,자율공상축구탐구만화,조석,조석,완결,스포츠,"월드컵, 유럽리그, 국내리그 등 축구라면 가리지 않고 모두 그려보는 자율축구탐구만화...",네이버 웹툰,0.628253
1454,취업의소리,조석,워니,완결,개그,"조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.500000
1042,우리들은 푸르다,문택수,문택수,완결,개그,그들과 함께라면 일상도 개그가 된다..! 범상치않은 그들이 모인 연파랑고 1학년 3...,네이버 웹툰,0.196250


### 삼국지톡

In [42]:
get_recomm('삼국지톡')

,title,writer,illustrator,day,genre,story,platform,similarity
1276,삵의 발톱,손영완,손영완,완결,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.474685
1293,비흔,정재한,황영찬,완결,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.465540
1016,천국의 신화 6부,올댓스토리,이현세,완결,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.464971
763,조선방랑야사,가현,가현,완결,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.463401
1177,문아,팬마,팬마,완결,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.462546
1109,오성X한음,유승진,유승진,완결,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성 한음...",네이버 웹툰,0.462198
900,왕으로 살다,네스티캣,네스티캣,완결,시대극,살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.461678
632,봉이 김선달,양우석,제피가루,완결,시대극,평양에서 훈장으로 소일하고 있던 봉이 김선달.평안감사로 온 조덕영의 폭정에 신음하는...,네이버 웹툰,0.461550
1387,흐드러지다,연제원,연제원,완결,시대극,"지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.461429
1455,태왕광개토,청안랑,김인호,완결,시대극,고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.461426


In [43]:
get_recomm('삼국지톡',w_author=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1276,삵의 발톱,손영완,손영완,완결,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.371448
1293,비흔,정재한,황영찬,완결,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.364293
1016,천국의 신화 6부,올댓스토리,이현세,완결,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.363847
763,조선방랑야사,가현,가현,완결,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.362619
1177,문아,팬마,팬마,완결,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.361949
1109,오성X한음,유승진,유승진,완결,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성 한음...",네이버 웹툰,0.361677
900,왕으로 살다,네스티캣,네스티캣,완결,시대극,살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.361271
632,봉이 김선달,양우석,제피가루,완결,시대극,평양에서 훈장으로 소일하고 있던 봉이 김선달.평안감사로 온 조덕영의 폭정에 신음하는...,네이버 웹툰,0.361170
1387,흐드러지다,연제원,연제원,완결,시대극,"지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.361076
1455,태왕광개토,청안랑,김인호,완결,시대극,고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.361073


In [44]:
get_recomm('삼국지톡',w_author=1.0,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1276,삵의 발톱,손영완,손영완,완결,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.263023
946,조선왕조실톡,무적핑크,무적핑크,완결,개그,"어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.261298
1293,비흔,정재한,황영찬,완결,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.254633
1016,천국의 신화 6부,올댓스토리,이현세,완결,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.254110
763,조선방랑야사,가현,가현,완결,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.252671
1177,문아,팬마,팬마,완결,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.251886
1109,오성X한음,유승진,유승진,완결,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성 한음...",네이버 웹툰,0.251567
1230,실질객관영화,무적핑크,무적핑크,완결,개그,10층에서 뛰어내렸는데 어떻게 살아있을 수 있지?정말 히어로는 무조건 이길 수 있을...,네이버 웹툰,0.251274
1329,실질객관동화,무적핑크,무적핑크,완결,개그,동화를 객관적인 시각에서 보면 어떻게 될까?생소한 시각으로 동화 다시 보기,네이버 웹툰,0.251130
900,왕으로 살다,네스티캣,네스티캣,완결,시대극,살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.251090


In [46]:
get_recomm('삼국지톡',noauthor=True,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
1276,삵의 발톱,손영완,손영완,완결,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.526047
1293,비흔,정재한,황영찬,완결,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.509266
1016,천국의 신화 6부,올댓스토리,이현세,완결,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.508221
763,조선방랑야사,가현,가현,완결,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.505341
1177,문아,팬마,팬마,완결,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.503771
1109,오성X한음,유승진,유승진,완결,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성 한음...",네이버 웹툰,0.503133
900,왕으로 살다,네스티캣,네스티캣,완결,시대극,살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.502180
632,봉이 김선달,양우석,제피가루,완결,시대극,평양에서 훈장으로 소일하고 있던 봉이 김선달.평안감사로 온 조덕영의 폭정에 신음하는...,네이버 웹툰,0.501945
1387,흐드러지다,연제원,연제원,완결,시대극,"지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.501723
1455,태왕광개토,청안랑,김인호,완결,시대극,고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.501717


In [45]:
get_recomm('삼국지톡',w_author=1.3,w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
946,조선왕조실톡,무적핑크,무적핑크,완결,개그,"어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.322526
1230,실질객관영화,무적핑크,무적핑크,완결,개그,10층에서 뛰어내렸는데 어떻게 살아있을 수 있지?정말 히어로는 무조건 이길 수 있을...,네이버 웹툰,0.315074
1329,실질객관동화,무적핑크,무적핑크,완결,개그,동화를 객관적인 시각에서 보면 어떻게 될까?생소한 시각으로 동화 다시 보기,네이버 웹툰,0.314967
1296,경운기를 탄 왕자님,무적핑크,무적핑크,완결,개그,"실질객관동화 무적핑크 작가의 신작!싹수 노란 남자의 농사 정복기, 오빤 강남 농부 ...",네이버 웹툰,0.314398
1276,삵의 발톱,손영완,손영완,완결,시대극,"중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.195556
1293,비흔,정재한,황영찬,완결,시대극,"잊혀진 나라 백제, 그리고 싸울아비.패자의 역사 뒤로 전설이 되어 사라진 검은 이리...",네이버 웹툰,0.189318
1016,천국의 신화 6부,올댓스토리,이현세,완결,시대극,"10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.188930
763,조선방랑야사,가현,가현,완결,시대극,한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.187859
1177,문아,팬마,팬마,완결,시대극,평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.187276
1109,오성X한음,유승진,유승진,완결,시대극,"영길리국에 셜록과 왓슨이 있다면 조선에는 이들이 있다!조선판 수사추리극, 오성 한음...",네이버 웹툰,0.187038
